In [8]:
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"
# отключение предупреждения

#os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold

In [9]:
import os 
import pathlib 
import numpy as np
from glob import glob

import shutil
import random

In [10]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
print("работаем на устройстве: ", DEVICE)
print('Версия torch:           ', torch.__version__)
print("версия  cuDNN:          ", torch.backends.cudnn.version())

True
работаем на устройстве:  cuda
Версия torch:            2.7.1+cu126
версия  cuDNN:           91100


In [11]:
# Текущее положение
path_1 = os.getcwd()
path_1

'/home/maksim/develops/python/projects/ML/NLP/imdb'

In [24]:
# Пути к директориям с данными

train_dir_1 = os.path.join(path_1, 'dataset2/train/')
test_dir_1 = os.path.join(path_1, 'dataset2/test/')

train_dir_2 = os.path.join(path_1, 'dataset3/train/')

dataset_new = os.path.join(path_1, 'data/dataset_new/')

In [13]:
# Расположение файла для кодировки слов
path_vocab = 'imdb.vocab'

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms


In [ ]:
# Загрузка данных
train_directory_1 = train_dir_1
train_directory_2 = train_dir_2



In [ ]:
import os
from utils.utils import transform_text

def load_dataset(base_path):
    data = []
    labels = []
    # Предположим, что каждая папка — это рейтинг, например "1", "2", ..., "10"
    for rating_folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, rating_folder)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                file_path = os.path.join(folder_path, filename)
                if os.path.isfile(file_path):
                    # Преобразуем текст файла в числовой вектор
                    vector = transform_text(file_path, path_vocab)  # ваша функция преобразования
                    data.append(vector)
                    labels.append(int(rating_folder))  # метка - рейтинг из имени папки
    
    return data, labels

# Загрузка данных
base_path = dataset_new
data, labels = load_dataset(base_path)

# Теперь можно использовать data и labels для обучения модели
# Например, вывести размеры
print(f"Всего отзывов: {len(data)}")
print(f"Пример номера метки и длины текста: {labels[0]}, {len(data[0])}")


Всего отзывов: 42927
Пример номера метки и длины текста: 1, 110


In [30]:
# Просмотр первых нескольких примеров
for i in range(5):  # первые 5 отзывов
    print(f"Отзыв #{i+1}:")
    print("Вектор:", data[i])
    print("Метка (рейтинг):", labels[i])
    print("-" * 40)


Отзыв #1:
Вектор: [569, 8, 1, 747, 2992, 0, 0, 9, 12, 2165, 51, 9, 1202, 55, 3558, 14, 0, 250, 108, 1, 0, 2, 109, 3, 0, 9, 842, 5, 65, 85, 4, 29, 3, 811, 0, 2862, 4963, 0, 60, 6350, 25191, 8, 4436, 74439, 2, 1, 4408, 195, 0, 653, 2, 0, 35, 3, 1446, 0, 0, 9, 216, 304, 1, 16, 8, 1, 0, 1, 1024, 12, 2095, 0, 216, 29, 102, 68, 0, 1, 10864, 772, 91, 1838, 1, 2046, 53, 0, 0, 13, 1165, 17655, 1042, 8, 24, 0, 8, 1, 128, 21, 89, 0, 582, 126, 0, 582, 126, 0, 2202, 154, 0, 0, 0, 88467]
Метка (рейтинг): 1
----------------------------------------
Отзыв #2:
Вектор: [3065, 14, 109, 0, 5, 21, 31, 1, 3507, 300, 745, 485, 5607, 0, 0, 6, 54, 50, 4, 3, 2055, 2057, 41, 35, 109, 21692, 15, 0, 10, 623, 907, 4, 3, 18, 497, 130, 15, 3, 6586, 833, 2, 259, 326, 4348, 35, 0, 3, 0, 507, 5, 26, 984, 360, 8, 5292, 3044, 23, 1, 3981, 4, 10, 3112, 0, 0, 7587, 44, 607, 130, 19, 3, 952, 1155, 202, 182, 0, 36, 52, 1026, 5, 14115, 2271, 5, 9103, 36, 34, 67, 1097, 1, 3134, 0, 9, 253, 115, 11, 39, 2327, 271, 14, 1735, 5, 138

In [28]:
import pickle

# Допустим, data - список с векторами, labels - список меток
with open('data/dataset.pkl', 'wb') as f:
    pickle.dump((data, labels), f)

# Загрузка датасета из файла
with open('data/dataset.pkl', 'rb') as f:
    data_loaded, labels_loaded = pickle.load(f)


In [32]:
import pickle

with open('data/dataset.pkl', 'rb') as f:
    data_loaded, labels_loaded = pickle.load(f)

for i in range(5):
    print(f"Отзыв #{i+1}:")
    print("Вектор:", data_loaded[i])
    print("Метка (рейтинг):", labels_loaded[i])
    print("-" * 40)


Отзыв #1:
Вектор: [569, 8, 1, 747, 2992, 0, 0, 9, 12, 2165, 51, 9, 1202, 55, 3558, 14, 0, 250, 108, 1, 0, 2, 109, 3, 0, 9, 842, 5, 65, 85, 4, 29, 3, 811, 0, 2862, 4963, 0, 60, 6350, 25191, 8, 4436, 74439, 2, 1, 4408, 195, 0, 653, 2, 0, 35, 3, 1446, 0, 0, 9, 216, 304, 1, 16, 8, 1, 0, 1, 1024, 12, 2095, 0, 216, 29, 102, 68, 0, 1, 10864, 772, 91, 1838, 1, 2046, 53, 0, 0, 13, 1165, 17655, 1042, 8, 24, 0, 8, 1, 128, 21, 89, 0, 582, 126, 0, 582, 126, 0, 2202, 154, 0, 0, 0, 88467]
Метка (рейтинг): 1
----------------------------------------
Отзыв #2:
Вектор: [3065, 14, 109, 0, 5, 21, 31, 1, 3507, 300, 745, 485, 5607, 0, 0, 6, 54, 50, 4, 3, 2055, 2057, 41, 35, 109, 21692, 15, 0, 10, 623, 907, 4, 3, 18, 497, 130, 15, 3, 6586, 833, 2, 259, 326, 4348, 35, 0, 3, 0, 507, 5, 26, 984, 360, 8, 5292, 3044, 23, 1, 3981, 4, 10, 3112, 0, 0, 7587, 44, 607, 130, 19, 3, 952, 1155, 202, 182, 0, 36, 52, 1026, 5, 14115, 2271, 5, 9103, 36, 34, 67, 1097, 1, 3134, 0, 9, 253, 115, 11, 39, 2327, 271, 14, 1735, 5, 138

In [39]:
import os
import random
import pickle




# функция загрузки словаря для кодировки слов из файла
def vocab_read(path_vocab):
    with open(path_vocab, "r") as f1:
        line = f1.read().splitlines()
    return line

# загрузка словаря и просмотр первых 10 слов
vocab = vocab_read(path_vocab)
print(vocab[:10])
n =len(vocab)
print(n)
vocab = vocab[:39998]
print(len(vocab))


# Параметры модели и тренировки
input_size = len(vocab) + 2  # Предполагаемый размер словаря (число уникальных слов)

hidden_size = 128
num_classes = 8  # Два класса: положительный и отрицательный отзыв
num_epochs = 10
batch_size = 32
learning_rate = 0.001

len_text = 280  # длина текста (клоичество слов) используемая для оценки 








# vocab - список слов словаря
# len_text - фиксированная длина текста (число слов)
# base_path - папка с разбитыми по рейтингам папками и текстами

def transform_text(text1, vocab, len_text, rand=False):
    mass = [] 
    str1 =''    
    fl = False
    list_1 = [0, 1]  # из вашего кода, для rand

    for ch in text1:
        if len(mass) > len_text - 1:  # если слов больше чем нужно, выходим
            return mass
        if ch != ' ':
            str1 = str1 + ch
        if ch == ' ':
            if str1 != '':                      
                fl = False                                                            
                for i in range(len(vocab)):
                    if str1.lower() == vocab[i].lower():
                        fl = True
                        random_choice = random.choice(list_1)
                        if rand == True and random_choice == 0:
                            mass.append(1)  # РАНДОМНО заполняем единицей
                            break
                        mass.append(i + 2)
                        str1 = ''
                        break                                                 
                if fl == False:  # если слово не найдено, заменяем нулями
                    mass.append(0)
                    str1 = ''        
    # Проверка последнего слова
    if str1 != '':
        fl = False
        for i in range(len(vocab)):
            if str1.lower() == vocab[i].lower():
                fl = True
                mass.append(i + 2)
                break                                                 
        if fl == False:
            mass.append(0)

    # Дополнение нулями слева, если слов меньше заданной длины
    while len(mass) < len_text:
        mass.insert(0, 0)
        
    return mass


def create_balanced_dataset(base_path, vocab, len_text, rand=False):
    classes = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]
    class_counts = {}

    # Считаем количество файлов в каждом классе
    for cl in classes:
        folder_path = os.path.join(base_path, cl)
        count = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
        class_counts[cl] = count

    # Минимальный размер класса
    min_count = min(class_counts.values())
    print(f"Минимальный размер класса: {min_count}")

    data = []
    labels = []

    for cl in classes:
        folder_path = os.path.join(base_path, cl)
        files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        # Берём случайные (или первые) min_count файлов
        if len(files) > min_count:
            files = random.sample(files, min_count)
        else:
            files = files[:min_count]

        for file_name in files:
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
            vec = transform_text(text, vocab, len_text, rand=rand)
            data.append(vec)
            labels.append(int(cl))  # если имя папки - число рейтинга

    return data, labels

# Путь к папке с датасетом, словарь и длина текста должны быть определены
base_path = dataset_new

# vocab = [...]  # список слов вашего словаря
len_text = 280  # например

data, labels = create_balanced_dataset(base_path, vocab, len_text, rand=False)

# Сохраняем в pickle
with open('balanced_dataset.pkl', 'wb') as f:
    pickle.dump((data, labels), f)

print("Датасет сохранён в 'balanced_dataset.pkl'")


['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this']
89527
39998
Минимальный размер класса: 4151
Датасет сохранён в 'balanced_dataset.pkl'


In [40]:
import pickle

# Предположим, что у вас есть переменные:
# data  — список векторов (числовых представлений отзывов)
# labels — список меток (рейтингов)

# Сохраняем в файл 'dataset.pkl'
with open('dataset.pkl', 'wb') as f:
    pickle.dump((data, labels), f)

print("Датасет успешно сохранён в 'dataset.pkl'")


with open('dataset.pkl', 'rb') as f:
    data_loaded, labels_loaded = pickle.load(f)


print(f"Загружено отзывов: {len(data_loaded)}")
print(f"Загружено меток: {len(labels_loaded)}")


Датасет успешно сохранён в 'dataset.pkl'
Загружено отзывов: 33208
Загружено меток: 33208


In [ ]:
import os
import random
import pickle

def transform_text(text1, vocab, len_text, rand=False):
    mass = [] 
    str1 =''    
    fl = False
    list_1 = [0, 1]

    for ch in text1:
        if len(mass) > len_text - 1:
            return mass
        if ch != ' ':
            str1 += ch
        if ch == ' ':
            if str1 != '':
                fl = False
                for i in range(len(vocab)):
                    if str1.lower() == vocab[i].lower():
                        fl = True
                        random_choice = random.choice(list_1)
                        if rand and random_choice == 0:
                            mass.append(1)
                            break
                        mass.append(i + 2)
                        str1 = ''
                        break
                if not fl:
                    mass.append(0)
                    str1 = ''
    # Проверка последнего слова
    if str1 != '':
        fl = False
        for i in range(len(vocab)):
            if str1.lower() == vocab[i].lower():
                fl = True
                mass.append(i + 2)
                break
        if not fl:
            mass.append(0)

    while len(mass) < len_text:
        mass.insert(0, 0)
    return mass


# def create_balanced_and_test_datasets(base_path, vocab, len_text, rand=False):
#     classes = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]
#     class_counts = {}

#     # Подсчёт количества файлов в каждом классе
#     for cl in classes:
#         folder_path = os.path.join(base_path, cl)
#         count = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
#         class_counts[cl] = count

#     min_count = min(class_counts.values())
#     print(f"Минимальное количество примеров на класс: {min_count}")

#     balanced_data = []
#     balanced_labels = []
#     test_data = []
#     test_labels = []

#     for cl in classes:
#         folder_path = os.path.join(base_path, cl)
#         files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
#         random.shuffle(files)

#         # Выбираем min_count файлов для сбалансированного датасета
#         balanced_files = files[:min_count]
#         # Остальные — для тестового датасета
#         test_files = files[min_count:]

#         # Обработка balanced
#         for file_name in balanced_files:
#             file_path = os.path.join(folder_path, file_name)
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 text = f.read()
#             vec = transform_text(text, vocab, len_text, rand=rand)
#             balanced_data.append(vec)
#             balanced_labels.append(int(cl))

#         # Обработка test
#         for file_name in test_files:
#             file_path = os.path.join(folder_path, file_name)
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 text = f.read()
#             vec = transform_text(text, vocab, len_text, rand=rand)
#             test_data.append(vec)
#             test_labels.append(int(cl))

#     return balanced_data, balanced_labels, test_data, test_labels


# Путь к датасету
base_path = dataset_new
# vocab = [...]  # ваш список слов
len_text = 280

balanced_data, balanced_labels, test_data, test_labels = create_balanced_and_test_datasets(base_path, vocab, len_text, rand=False)

# Сохраняем оба датасета по отдельности
with open('balanced_dataset.pkl', 'wb') as f:
    pickle.dump((balanced_data, balanced_labels), f)

with open('test_dataset.pkl', 'wb') as f:
    pickle.dump((test_data, test_labels), f)

print("Сбалансированный датасет сохранён в 'balanced_dataset.pkl'")
print("Тестовый датасет сохранён в 'test_dataset.pkl'")


Минимальное количество примеров на класс: 4151
Сбалансированный датасет сохранён в 'balanced_dataset.pkl'
Тестовый датасет сохранён в 'test_dataset.pkl'


In [1]:
def inv1(filename):
    '''
    Функция для получения рейтинга отзыва и номера (названия) файла по имени файла.
    Ожидается, что имя файла имеет формат "номер_рейтинг.txt", например "2429_1.txt".
    Возвращает кортеж: (рейтинг как строка, имя файла без расширения)
    '''
    # Убираем расширение файла
    base_name = filename.split('.')[0]
    # Разделяем по подчёркиванию
    parts = base_name.split('_')
    if len(parts) < 2:
        return None, None  # Если формат некорректный
    
    rating = parts[-1]  # Рейтинг — часть после последнего подчёркивания
    file_number = '_'.join(parts[:-1])  # Всё до рейтинга — номер файла (может содержать подчёркивания)
    
    return rating, file_number


In [3]:
rating, file_num = inv1("2429_10.txt")
print(rating)   # '1'
print(file_num) # '2429'


10
2429


In [43]:
import pickle

# Допустим, у вас есть:
# balanced_data, balanced_labels — сбалансированный датасет и метки
# test_data, test_labels — тестовый датасет и метки

# Сохраняем сбалансированный датасет
with open('balanced_dataset.pkl', 'wb') as f:
    pickle.dump((balanced_data, balanced_labels), f)

# Сохраняем тестовый датасет
with open('test_dataset.pkl', 'wb') as f:
    pickle.dump((test_data, test_labels), f)

print("Датасеты успешно сохранены")


import pickle

# Загружаем сбалансированный датасет
with open('balanced_dataset.pkl', 'rb') as f:
    balanced_data_loaded, balanced_labels_loaded = pickle.load(f)

# Загружаем тестовый датасет
with open('test_dataset.pkl', 'rb') as f:
    test_data_loaded, test_labels_loaded = pickle.load(f)

print(f"Сбалансированных примеров: {len(balanced_data_loaded)}")
print(f"Тестовых примеров: {len(test_data_loaded)}")



Датасеты успешно сохранены
Сбалансированных примеров: 33208
Тестовых примеров: 9719


In [44]:
import pickle

with open('data/balanced_dataset.pkl', 'rb') as f:
    data_loaded, labels_loaded = pickle.load(f)

for i in range(5):
    print(f"Отзыв #{i+1}:")
    print("Вектор:", data_loaded[i])
    print("Метка (рейтинг):", labels_loaded[i])
    print("-" * 40)


Отзыв #1:
Вектор: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 313, 0, 9765, 307, 7, 2, 0, 313, 3331, 17, 9, 2, 0, 0, 8, 64, 45, 158, 6, 83, 16, 2, 195, 290, 313, 3331, 101, 533, 49, 5, 2, 103, 24, 409, 6, 27, 2857, 6, 2, 103, 9, 2, 195, 2303, 3, 4913, 8890, 7, 9, 8, 0, 12, 225, 76, 125, 0, 0, 0, 7, 54, 114, 6, 6980, 11, 17, 0, 39, 682, 99, 160, 0, 2, 111, 7, 1291, 6, 0, 254, 2, 238, 16, 2, 0, 1147, 61, 902, 51, 38, 4, 0, 10117, 5, 4, 684, 236, 4441, 6, 27, 4, 11056, 32, 749, 2, 685, 0, 11, 17, 163, 70, 233, 38, 138, 33, 0, 135, 156, 10, 6629, 6, 4, 1155, 115, 1212, 260, 6, 796, 0, 1315, 207, 50, 0, 506, 4116, 190, 65, 1603, 131, 9, 0, 1315, 45, 13984, 20615, 383, 202, 2, 9003, 5, 5243, 3, 0, 3, 22, 68, 33277, 30, 130, 4, 0, 211, 4351, 202, 40, 0, 0, 11, 201, 45, 10151, 396, 6, 2, 1492, 5, 11420, 0, 268, 15, 2, 363, 0, 3583, 401, 987, 185, 

In [38]:
# функция загрузки словаря для кодировки слов из файла
def vocab_read(path_vocab):
    with open(path_vocab, "r") as f1:
        line = f1.read().splitlines()
    return line

# загрузка словаря и просмотр первых 10 слов
vocab = vocab_read(path_vocab)
print(vocab[:10])
n =len(vocab)
print(n)
vocab = vocab[:39998]
print(len(vocab))

['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this']
89527
39998


In [36]:
# Параметры модели и тренировки
input_size = len(vocab) + 2  # Предполагаемый размер словаря (число уникальных слов)

hidden_size = 128
num_classes = 8  # Два класса: положительный и отрицательный отзыв
num_epochs = 10
batch_size = 32
learning_rate = 0.001

len_text = 280  # длина текста (клоичество слов) используемая для оценки 


# Определение числа фолдов
k_folds = 5

In [37]:
print(len(vocab))
input_size

39998


40000

In [17]:
list_1 = list(range(12))
print(list_1)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [18]:
# функция преобразования текста в числа

def transform_text(text1, vocab=vocab, len_text=len_text, rand=False):
    mass = [] 
    str1 =''    
    for ch in text1:
        if len(mass)>len_text-1: # если слов больше чем нужно, выходим
            return mass
        if ch !=' ':
            str1 = str1 + ch
        if ch ==' ':
            if str1 !='':                      
                fl = False                                                            
                for i in range(len(vocab)):
                    if str1.lower() == vocab[i].lower():
                        fl = True
                        random_choice = random.choice(list_1)
                        if rand==True:
                            if random_choice == 0:
                                mass.append(1)  #  РАНДОМНО заполняем единицей
                                break
                        mass.append(i+2)
                        str1 =''
                        break                                                 
                if fl == False: #  если слово не найдено, заменяем нулями
                    mass.append(0)
                    str1 =''        
    for i in range(len(vocab)): # Проверяем последнее слово
        if str1.lower() == vocab[i].lower():
            fl = True
            mass.append(i+2)
            str1 =''
            break                                                 
    if fl == False: #  если слово не найдено, заменяем нулями
        mass.append(0)
        str1 =''

    if len(mass)<len_text:  #  если слов меньше чем нужно, добавляем нулями.
        while len(mass)<len_text:
            mass.insert(0, 0)
            str1 =''        
        
    return mass

In [ ]:
dataset_new

In [ ]:
def rating(num, data):
    text_example = transform_text(data[num], vocab, 15)
    print(text_example)
    text_example1 = data[num]
    print(text_example1[:150])
    print('РЕЙТИНГ ФИЛЬМА: ', test_labels_1[num])

In [30]:
rating(20000)

[10, 423, 82, 11, 17, 978, 4, 0, 406, 43, 87, 6, 27, 3152, 9]
I went into this movie expecting a thoughtfull piece about how to be accepted in culture and I wound up blowing $8.50 on a 10 minute fart joke and a w
РЕЙТИНГ ФИЛЬМА:  3


In [31]:
txt = 'the end and and fgh I the open the open end'

In [32]:
text_example = transform_text(txt, vocab, 15)
text_example

[0, 0, 0, 0, 2, 129, 3, 3, 0, 10, 2, 940, 2, 940, 129]

In [33]:
text_example = transform_text(test_data_1[20000], vocab, 15)
text_example

[10, 423, 82, 11, 17, 978, 4, 0, 406, 43, 87, 6, 27, 3152, 9]

In [34]:
test_data_1[20000]

"I went into this movie expecting a thoughtfull piece about how to be accepted in culture and I wound up blowing $8.50 on a 10 minute fart joke and a whole bunch of fake accents. Sorry, Jeff, but if you're going for the whole cult thing, you gotta a while to go."

In [ ]:
data_tensor = torch.tensor(text_example)

In [36]:
data_tensor

tensor([  10,  423,   82,   11,   17,  978,    4,    0,  406,   43,   87,    6,
          27, 3152,    9])

In [37]:
def preprocess_data(data, labels):
    processed_data = []
    
    for text1 in data:  
        text_list = transform_text(text1)
        processed_data.append(text_list)

    data_tensor = torch.tensor(processed_data)
    labels_tensor = torch.tensor(labels)
    return data_tensor, labels_tensor

In [38]:
list1 = [1, 2, 3]
list2 = [4, 5, 6]

list1 += list2
print(list1)

[1, 2, 3, 4, 5, 6]


In [39]:
def preprocess_1(data_1, labels_1, data_2, labels_2):
    processed_data = []
    labels = []
    
    for text1 in data_1:  
        text_list = transform_text(text1)
        processed_data.append(text_list)      
    labels +=  labels_1  
    
    for text1 in data_2:  
        text_list = transform_text(text1, rand=True)
        processed_data.append(text_list)      
    labels +=  labels_2  

    data_tensor = torch.tensor(processed_data)
    labels_tensor = torch.tensor(labels)
    return data_tensor, labels_tensor
    
    
    
    

In [40]:
def rating_test(num):
    
    text_example = transform_text(test_data[num], vocab, 15)
    print(text_example)
    text_example1 = test_data[num]
    print(text_example1[:150])
    print('РЕЙТИНГ ФИЛЬМА: ', test_labels[num])
    
    print('Новые значения ')
    print(test_data_tensor[num][:15])
    print(test_labels_tensor[num])

In [ ]:
import matplotlib.pyplot as plt

def train_model(model, train_loader, val_loader, num_epochs=5, save_model_path='model_1.pth'):
    train_losses = []  # Список для сохранения значений функции потерь на тренировочном наборе
    val_losses = []  # Список для сохранения значений функции потерь на валидационном наборе
    train_accuracies = []  # Список для сохранения значений точности на тренировочном наборе
    val_accuracies = []  # Список для сохранения значений точности на валидационном наборе

    best_val_accuracy = 0

    for epoch in range(num_epochs):
        # Обучение модели...

        # Вычисление точности и потерь на тренировочном и валидационном наборах
        train_accuracy, train_loss = compute_accuracy_and_loss(model, train_loader, criterion)
        val_accuracy, val_loss = compute_accuracy_and_loss(model, val_loader, criterion)

        # Сохранение значений точности и потерь
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)

        print(f"Epoch {epoch + 1}/{num_epochs} | "
              f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

        # Проверка, является ли текущая модель лучшей
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            # Сохранение модели
            torch.save(model.state_dict(), save_model_path)

    # Построение графиков точности и потерь
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Функция для вычисления точности и потерь
def compute_accuracy_and_loss(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            total_loss += loss.item()

    accuracy = total_correct / total_samples
    loss = total_loss / len(data_loader)

    return accuracy, loss


In [41]:
def rating_train(num):
    
    text_example = transform_text(train_data[num], vocab, 15)
    print(text_example)
    text_example1 = train_data[num]
    print(text_example1[:150])
    print('РЕЙТИНГ ФИЛЬМА: ', train_labels[num])
    
    print('Новые значения ')
    print(train_data_tensor[num][:15])
    print(train_labels_tensor[num])

In [42]:
test_data1, test_labels1 = preprocess_data(test_data_1, test_labels_1)




In [43]:
torch.save(test_data1, 'test_data4.pt')
torch.save(test_labels1, 'test_labels4.pt')

In [58]:
train_data1, train_labels1 = preprocess_1(train_data_1, train_labels_1, train_data_2, train_labels_2)

torch.save(train_data1, 'train_data3.pt')
torch.save(train_labels1, 'train_labels3.pt')

In [59]:
train_data_tensor = torch.load('train_data3.pt')
train_labels_tensor = torch.load('train_labels3.pt')

In [60]:
len(train_data_tensor)

65200

In [61]:
len(train_labels_tensor)

65200

In [62]:
print(train_data_tensor[50000])
print(train_labels_tensor[50000])

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [53]:
test_data_tensor = torch.load('test_data3.pt')
test_labels_tensor = torch.load('test_labels3.pt')

In [57]:

print(test_data_tensor[10])
print(test_labels_tensor[10])

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,   10,  187,  161,  102,    5,   28,   50,  213,    9,   11,    0,
           2,   63,    7,  413,    0,    2,  111,    7,   14, 4407,   14,    4,
           0,    3,    2,  223,    7,   

In [45]:
# Загрузка данных из файлов
train_data_tensor = torch.load('train_data3.pt')
train_labels_tensor = torch.load('train_labels3.pt')


test_data_tensor = torch.load('test_data3.pt')
test_labels_tensor = torch.load('test_labels3.pt')

In [32]:
'''
train_data1, train_labels1 = preprocess_data(train_data, train_labels)

test_data1, test_labels1 = preprocess_data(test_data, test_labels)
'''

'\ntrain_data1, train_labels1 = preprocess_data(train_data, train_labels)\nval_data1, val_labels1 = preprocess_data(val_data, val_labels)\ntest_data1, test_labels1 = preprocess_data(test_data, test_labels)\n'

In [33]:
# Сохранение данных в файлы
'''
torch.save(train_data1, 'train_data1.pt')
torch.save(train_labels1, 'train_labels1.pt')
torch.save(val_data1, 'val_data1.pt')
torch.save(val_labels1, 'val_labels1.pt')
torch.save(test_data1, 'test_data1.pt')
torch.save(test_labels1, 'test_labels1.pt')
'''

"\ntorch.save(train_data1, 'train_data1.pt')\ntorch.save(train_labels1, 'train_labels1.pt')\ntorch.save(val_data1, 'val_data1.pt')\ntorch.save(val_labels1, 'val_labels1.pt')\ntorch.save(test_data1, 'test_data1.pt')\ntorch.save(test_labels1, 'test_labels1.pt')\n"

In [34]:


# Загрузка данных из файлов
train_data_tensor = torch.load('train_data1.pt')
train_labels_tensor = torch.load('train_labels1.pt')
val_data_tensor = torch.load('val_data1.pt')
val_labels_tensor = torch.load('val_labels1.pt')
test_data_tensor = torch.load('test_data1.pt')
test_labels_tensor = torch.load('test_labels1.pt')

In [35]:
# Преобразование меток классов
'''
train_labels_tensor = [label - 1 if label <= 4 else label - 3 for label in train_labels_tensor]
val_labels_tensor = [label - 1 if label <= 4 else label - 3 for label in val_labels_tensor]
test_labels_tensor = [label - 1 if label <= 4 else label - 3 for label in test_labels_tensor]
'''

'\ntrain_labels_tensor = [label - 1 if label <= 4 else label - 3 for label in train_labels_tensor]\nval_labels_tensor = [label - 1 if label <= 4 else label - 3 for label in val_labels_tensor]\ntest_labels_tensor = [label - 1 if label <= 4 else label - 3 for label in test_labels_tensor]\n'

In [36]:
train_labels_tensor = torch.tensor(train_labels_tensor)
train_labels_tensor = torch.where(train_labels_tensor <= 4, train_labels_tensor - 1, train_labels_tensor - 3)

val_labels_tensor = torch.tensor(val_labels_tensor)
val_labels_tensor = torch.where(val_labels_tensor <= 4, val_labels_tensor - 1, val_labels_tensor - 3)

test_labels_tensor = torch.tensor(test_labels_tensor)
test_labels_tensor = torch.where(test_labels_tensor <= 4, test_labels_tensor - 1, test_labels_tensor - 3)

C:\Users\drug1\AppData\Local\Temp\ipykernel_14116\3457568113.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(train_labels_tensor)
C:\Users\drug1\AppData\Local\Temp\ipykernel_14116\3457568113.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_labels_tensor = torch.tensor(val_labels_tensor)
C:\Users\drug1\AppData\Local\Temp\ipykernel_14116\3457568113.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels_tensor = torch.tensor(test_labels_tensor)


In [37]:
len(test_labels_tensor)

13312

In [38]:
len(val_labels_tensor)

4688

In [39]:
len(train_labels_tensor)

32000

In [40]:
train_labels[31000]

9

In [41]:
train_labels_tensor[31000]

tensor(6)

In [42]:
test_labels_tensor[6000]

tensor(7)

In [43]:
train_data[30000]

'It\'s remarkable and quite praiseworthy how writers and directors continue to make great movies out of one of the oldest and most (over)used story lines in cinema! "Dog Bite Dog" is basically not much more than just the simple story of an lone copper obsessively chasing a brilliant criminal, only Pou-Soi Cheang distinguishes his film from the rest by being extremely violent & relentless. This is unquestionably one of the grittiest and most uncompromising movies I\'ve ever seen, with an atmosphere of constant nihilism and characters that seem to come walking straight out of hell! Not even the installments in Chan-Wook Park\'s trilogy of vengeance (with the exception of "Oldboy", perhaps) or any other infamous Cat-III film ever released were as sadistic and brutal as some of the events depicted in "Dog Bite Dog". Pang is a young and ruthless Cambodian assassin who lands in the crowded streets of Hong Kong to eliminate the wife of an eminent judge in a restaurant. When the police arrives

In [44]:
train_data_tensor[30000]

tensor([   0, 1677,    2,  176,    0,   86,  893,    2,  872, 1685,    5,   95,
          84,  100,   45,    4,   27,    4,    1, 5439,    2,   88,    0,   62,
         398,    8,    0,    0, 3872,    0,    6,  669,   20,   73,   50,   71,
          41,    1,  588,   62,    4,   32, 4708,    0,    0, 3270,    3,  510,
           0,   61,    0,    0,    0,   24,   18,   35,    1,  349,   31,  109,
         556, 1099,    0,    0,   10,    6,    0,   27,    4,    1,    0,    2,
          88,    0,  100,  198,  122,    0,   15,   32,  810,    4, 1766,    0,
           2,  102,   11,  297,    5,  211, 1254,  791,   45,    4,    0,   20,
          54,    1,    0,    8,    0,    0, 2302,    4, 4445,    0,    1, 1345,
           4,    0,    0,   40,   98,   79, 3065,    0,   18,  122,  616,   68,
          13, 3876,    2, 1719,   13,   48,    4,    1,  665, 2320,    8,    0,
        3872,    0,    0,    6,    3,  180,    2, 4191,    0, 4824,   33, 5220,
           8,    1, 8372, 1884,    4, 25

In [45]:
# Датасеты
train_dataset = torch.utils.data.TensorDataset(train_data_tensor.clone().detach(), train_labels_tensor.clone().detach())
val_dataset = torch.utils.data.TensorDataset(val_data_tensor.clone().detach(), val_labels_tensor.clone().detach())
test_dataset = torch.utils.data.TensorDataset(test_data_tensor.clone().detach(), test_labels_tensor.clone().detach())


In [46]:
# Создание итераторов для загрузки данных

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
